In [1]:
import numpy as np #linear algebra
import pandas as pd #data processing 
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

# Importing into dataframe
# df_Title_Basics = pd.read_csv("./Prj_Data/DownLoadedData_Imdb/title.basics (1).tsv.gz", delimiter='\t', encoding='ANSI')
# df_Name_Basics = pd.read_csv("./Prj_Data/DownLoadedData_Imdb/name.basics (1).tsv.gz", delimiter='\t', encoding='ANSI')

# df_Generes_With_tconst = pd.read_excel("df_Generes_With_tconst.xlsx")
df_IMDB_MovieCatalog = pd.read_excel("df_IMDB_MovieCatalog.xlsx")
df_starsplayers_knownForMovie = pd.read_excel ('./Prj_Data/ImdbScrapingData/TheNumbers/output/df_starsplayers_knownForMovies.xlsx', sheet_name='Export')
df_starsplayers = pd.read_excel ('./Prj_Data/ImdbScrapingData/TheNumbers/input/PeopleMetaData.xlsx', sheet_name='Export')
# df_Principles = pd.read_csv("./Prj_Data/DownLoadedData_Imdb/title.principals (1).tsv.gz", delimiter='\t', encoding='ANSI')


FileNotFoundError: [Errno 2] No such file or directory: 'df_IMDB_MovieCatalog.xlsx'

In [ ]:
#_________________________________Inserting nconst and tconst into Numbers Meatadata actor tables________________________

In [ ]:
#1. Start by Narrowing the Basics and Principles tables down by only getting data that can be linked to our 
#main movies tconts/ tabls
df_IMDB_MovieCatalog_Short = df_IMDB_MovieCatalog[["tconst","titleyear"]]

In [ ]:
ActorswithMoviesWeCareAbout = df_IMDB_MovieCatalog_Short.merge(df_Principles, on="tconst", how="inner").merge(df_Name_Basics, on="nconst",how="inner")


In [ ]:
#2. Now that you have reduced the number of possible linking rows for our Star Players Table, link on name_IMBD
#  Remember this first requires you to make sure the names in IMDB (Robert Downey, JR vs Robert Downey J.R link up
# via updating the People metadata table manually cross referencing via eye and cut and paste)


In [ ]:
#get star actors connected to nconst
df_starsplayers_WithMixedNconst = df_starsplayers.merge(ActorswithMoviesWeCareAbout,left_on="Name_IMDB", right_on="primaryName", how="outer", indicator="left")
df_starsplayers_WithMixedNconst = df_starsplayers_WithMixedNconst.drop_duplicates()


In [ ]:
#------SAVE----Used to create cross reference for names in IMDB to names in Numbers (see to facilitate the above step)
# df_starsplayers_MixedNconst = df_starsplayers.merge(ActorswithMoviesWeCareAbout,left_on="Name_IMDB", right_on="primaryName", how="outer", indicator="outer")
# df_starsplayers_MixedNconst = df_starsplayers_MixedNconst.drop_duplicates()
# #get star actors connected to nconst

In [ ]:
# Make sure not missing any important people when inserting nconst
# df_starsplayers_MixedNconst_Null_nconst = df_starsplayers_MixedNconst[df_starsplayers_MixedNconst['nconst'].isnull()]
df_starsplayers_MixedNconst[df_starsplayers_MixedNconst['nconst'].isnull()].sort_values(by="Name")

In [ ]:
#DF 1 NCONST ONLY
df_StarActors_With_nconst = df_starsplayers_WithMixedNconst[['Rank','Name_IMDB', 'Name', 'Domestic Box Office', 'Movies', 'Average', 'Link', 'Contribution', 'BirthdayClean', 'nameyear', 'nconst']]
df_StarActors_With_nconst = df_StarActors_With_nconst.drop_duplicates()
df_StarActors_With_nconst = df_StarActors_With_nconst.dropna(subset=["Name"])

In [ ]:
df_StarActors_With_nconst.to_excel("df_StarActors_With_nconst.xlsx")

In [ ]:
#DF 2 NCONST AND TCONST
df_StarActors_With_nconst_tconst = df_starsplayers_WithMixedNconst[['Rank', 'Name_IMDB', 'Name', 'Domestic Box Office', 'Movies', 'Average',
       'Link', 'Contribution', 'BirthdayClean', 'nameyear', 'nconst', 'tconst']]
df_StarActors_With_nconst_tconst = df_StarActors_With_nconst_tconst.drop_duplicates()
df_StarActors_With_nconst_tconst = df_StarActors_With_nconst_tconst.dropna(subset=["Name"])

In [ ]:
df_StarActors_With_nconst_tconst.to_excel("df_StarActors_With_nconst_tconst.xlsx")

In [ ]:
#________________________CODE FOR CHECKING THE ABOVE FOR ERRORS,ETC

In [ ]:
df_StarActors_With_nconst[["Name","Contribution","nconst"]].sort_values(by="Name").groupby(["Name", "Contribution"]).count()

In [ ]:
# df_StarActors_With_nconst_tconst[["Name","Contribution","nconst","tconst"]].sort_values(by="Name").groupby(["Name", "Contribution"]).count()

df_StarActors_With_nconst_tconst[["Name","Contribution","tconst", "Movies"]].sort_values(by="Name").groupby(["Name", "Contribution"]).agg({'tconst':'count','Movies':'max'})


In [ ]:
#_________________________END________________________________________________________________________________________

In [ ]:
#_________________________insert tconstinto KnownforMoviesTable_________________________________________

In [129]:
df_starsplayers_knownForMovie["IMDB_XREF"] = df_starsplayers_knownForMovie["IMDB_XREF"].fillna("0")
df_starsplayers_knownForMovie["ReleaseYear"] = df_starsplayers_knownForMovie["ReleaseYear"].astype(str)

df_starsplayers_knownForMovie["ReleaseYear"] = df_starsplayers_knownForMovie["ReleaseYear"].apply(lambda x: str(x)[:-2])

df_starsplayers_knownForMovie["title_linktoIMDB"] = df_starsplayers_knownForMovie["IMDB_XREF"].str.title()

df_starsplayers_knownForMovie["titleyear_titlelinktoIMDB"] = df_starsplayers_knownForMovie["title_linktoIMDB"] + df_starsplayers_knownForMovie["ReleaseYear"]

In [131]:
df_IMDB_MovieCatalog["primaryTitle"] = df_IMDB_MovieCatalog["primaryTitle"].str.title()
df_IMDB_MovieCatalog["titleyear1"] = df_IMDB_MovieCatalog["primaryTitle"] + df_IMDB_MovieCatalog["startYear_str"].astype(str)

# df_IMDB_MovieCatalog["titleyear"] = df_IMDB_MovieCatalog.apply(lambda row : row["primaryTitle"] + str(row["startYear_str"]))

In [133]:
df_IMDB_MovieCatalog_Short = df_IMDB_MovieCatalog[[ "primaryTitle","titleyear1","tconst"]]

In [135]:
df_KnownForMerged = df_starsplayers_knownForMovie.merge(df_IMDB_MovieCatalog_Short, left_on="titleyear_titlelinktoIMDB" , right_on="titleyear1", how="left")

In [136]:
df_KnownForMerged.to_excel("df_StarActors_KnownForWithTconst.xlsx")

In [77]:
#_______________________________USED TO UPDATE TITLE MANUALLY in Star Play DF Titltes KNOWN FOR(500) DAYS OF SUMMER vs 500 Days of Summer 
df_KnownForMerged_nulls = df_KnownForMerged[["title_linktoIMDB","titleyear1","tconst"]][df_KnownForMerged["tconst"].isnull()]
df_KnownForMerged_nulls = df_KnownForMerged_nulls.drop_duplicates()

In [31]:
# code for checking for errors
df_IMDB_MovieCatalog[["titleyear1","tconst"]][df_IMDB_MovieCatalog["titleyear"].str.contains('12 Years A Slave')]

,titleyear,tconst
100,12 Years A Slave2013,tt2024544


In [ ]:
#___________________________________END________________________

In [ ]:
#________________________Remove Dups FROM Star Players Table... just asssume/Uses Primary Contributions_________________________________________

In [ ]:
df_Stars_no_dups = df_starsplayers.sort_values(by=["Name", "Contribution"])
df_Stars_no_dups1 = df_Stars_no_dups.drop_duplicates(subset="Name", keep="first")

In [ ]:
df_Stars_no_dups1.to_excel("df_StarActors_NoDups.xlsx")